# Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from keras.models import load_model
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential,load_model

from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

# Load Data Set

In [ ]:
Vehicle_Type = 'Electric_Vehicles'
Folder = 'Dataset/SampleData'

df = pd.read_csv(f'{Folder}/{Vehicle_Type}_Final_Training_Samples.csv')
print(len(df))

# Processsing Inputs and Labels

In [ ]:
df = df.values.tolist()
DataSet = np.array(df,dtype=float)

print('*******************************')

Tupple = np.shape(DataSet)                  #34011 Segments x 18 Features
Number_of_Records = Tupple[0]
Number_of_Features = Tupple[1]

print(f'Number_of_Records = {Number_of_Records}')
print(f'Number_of_Features = {Number_of_Features}')


#Processing Sample_Inputs without Target Feature

X = np.zeros((Number_of_Records, Number_of_Features - 1))
for t in range(Number_of_Records):
    X[t] = DataSet[t, :(Number_of_Features-1)]

print(f'Input_Size = {np.shape(X)}')

#Processings Sample_Lables with Target feature ('Fuel_Consumed')

y = np.zeros((Number_of_Records,))
for t in range(Number_of_Records):
    y[t] = DataSet[t][Number_of_Features-1]

print(f'Label_Size = {np.shape(y)}')


print('*******************************')

# Random Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f'Training_Input = {np.shape(X_train)}')
print(f'Training_Labels = {np.shape(y_train)}')

print(f'Testing_inputs = {np.shape(X_test)}')
print(f'Testing_Labels = {np.shape(y_test)}')

# Removing Time_Needed Feature

In [ ]:
# Saving Time Needed for predicting for samples with longer intervals
Time_needed = []

for i in X_test:
    Time_needed.append(i[1])

print(np.shape(Time_needed))

#Deleting Time_Needed before Training

X_train = np.delete(X_train, 1, 1)
X_test = np.delete(X_test, 1, 1)

print(np.shape(X_train))
print(np.shape(y_train))

T = np.shape(X_test) #For Plotting

print(T)

Length_of_TestData = T[0]

# ANN Model - Feed Forward

In [ ]:
def SimpleFeedForward_Model(number_of_epoch):
    
    # print(f'number_of_iteration = {number_of_epoch}')
    # FF_model = Sequential()
    #
    # FF_model.add(Dense(100,  input_shape=(Number_of_Features-2,), activation="sigmoid"))
    # FF_model.add(Dense(80, activation="sigmoid"))
    # FF_model.add(Dense(1, activation = "linear"))
    #
    # optimizer = keras.optimizers.SGD(lr=0.1)
    # def mean_error(y_test, y_pred):
    #     return tf.reduce_mean(y_pred - y_test)
    #
    # checkpoint = ModelCheckpoint(f'{Folder}/PredictionModels/Electric/FF_model-All-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5', verbose=1, monitor='val_loss',
    #                              save_best_only=True, mode='auto')
    #
    # FF_model.compile(loss='mse', optimizer = 'adam', metrics=['mae'])
    #
    #
    #
    # history = FF_model.fit(X_train, y_train,
    #     shuffle=True,
    #     epochs=number_of_epoch,
    #     batch_size=128,
    #                        # validation_split=0.05,
    #     # validation_data=(X_val,y_val),
    #     validation_data=(X_test, y_test),
    #     callbacks=[checkpoint],
    #     verbose=2)
    #
    # # print(FF_model.summary())
    #
    #
    #
    # print(history.history.keys())
    #

    #
    # #summarize history for loss
    # plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    # plt.title('Model Loss')
    # plt.ylabel('Loss = MSE')
    # plt.xlabel('Iteration')
    # plt.legend(['Train', 'Test'], loc='upper left')
    #
    # plt.show()

    # #load model
    FF_model = load_model(f'{Folder}/PredictionModels/Electric/FF_model-ALL-799-0.031906-0.031194.h5')

    print("MSE\t\tMAE")

    scores = FF_model.evaluate(X_train, y_train, verbose=2)
    print("Training Set Loss: {}".format(scores))

    # scores = FF_model.evaluate(X_val, y_val, verbose=2)
    # print("Validation Set Loss: {}".format(scores))

    scores = FF_model.evaluate(X_test, y_test, verbose=2)
    print("Testing Set Loss: {}".format(scores))

    y_pred = FF_model.predict(X_test, verbose=2, batch_size=64)

    return y_pred

# Decision Tree Model

In [ ]:
def DecisionTree_Model():
    # DT_regressor = DecisionTreeRegressor(max_depth= Number_of_Features,
    #                                   splitter = 'best',
    #                                   criterion="mse", min_samples_leaf=0.001)

    # DT_regressor.fit(X_train, y_train)

    # y_pred = DT_regressor.predict(X_train)

    # MSE = np.sum(((y_train - y_pred) ** 2) / len(y_train))
    # print(f'MSE on Train = {MSE}')

    # y_pred = DT_regressor.predict(X_test)

    # MSE = np.sum(((y_test - y_pred) ** 2) / len(y_test))
    # print(f'MSE on Test = {MSE}')

    # RMSE = np.sqrt(np.sum(((y_test - y_pred) ** 2) / len(y_test)))
    # print(f'RMSE = {RMSE}')

    # MAE = np.sum(abs(y_test - y_pred) / len(y_test))
    # print(f'MAE = {MAE}')

    filename = f'{Folder}/PredictionModels/Electric/finalized_DT_model.sav'
    #pickle.dump(DT_regressor, open(filename, 'wb'))

    ###load the model from disk
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(X_test)
    score = loaded_model.score(X_test, y_test)
    print(f'R^2 Score = {score}')


    return y_pred

# Linear Regression Model

In [ ]:
def LinearRegression_Model():

    # regression_model = LinearRegression(fit_intercept=True, normalize=True)
    # regression_model.fit(X_train, y_train)

    # # regression coefficients

    # print('Coefficients for each feature:')
    # print(regression_model.coef_)

    # R_Square = regression_model.score(X_test, y_test)  # Explained variance score: 1 is perfect prediction
    # print(f'R_Square = {R_Square}')

    # y_pred = regression_model.predict(X_train)

    # regression_model_mse = mean_squared_error(y_train, y_pred)
    # regression_model_mae = mean_absolute_error(y_train, y_pred)

    # print(f'regression_model_mse on Train = {regression_model_mse}')
    # print(f'regression_model_mae on Train = {regression_model_mae}')

    # y_pred = regression_model.predict(X_test)

    # regression_model_mse = mean_squared_error(y_test,y_pred)
    # regression_model_mae = mean_absolute_error(y_test,y_pred)

    # print(f'regression_model_mse on Test = {regression_model_mse}')
    # print(f'regression_model_mae on test = {regression_model_mae}')



    filename = f'{Folder}/PredictionModels/Electric/finalized_LR_model.sav'
    #pickle.dump(regression_model, open(filename, 'wb'))
    
    
    ###load the model from disk
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(X_test)
    score = loaded_model.score(X_test, y_test)
    print(f'R^2 Score = {score}')


    return y_pred

# Plotting Predicted vs Actual

In [ ]:
def Plotting_Pred_Actual(Model_Name,Actual_Values,PredictedValues,Length_of_TestData,Count):

    #Each iteration will plot results for 25 samples.
    
    initial = 0
    final = 25
    for initial in range(0,Length_of_TestData+1,25):
        Samples = list(range(initial,final,1))
    
        plt.grid(color='b', linestyle='--', linewidth=0.1)
        plt.plot(PredictedValues[initial:final],'*',color = 'k')
        plt.plot(Actual_Values[initial:final], '--',marker = 'o',color = 'r')
        plt.xlabel(f'Samples from {initial} to {final-1}',size = 20)#,weight='bold')
        xi = [i for i in range(0,len(Samples),1)]
        plt.xticks(xi,Samples,rotation=90)
    
        plt.ylabel('Energy Consumption',size=20)#,weight='bold')
        plt.title(f'Predicted Energy Consumption vs Actual Energy Consumption\n{Model_Name}',size=20)#,weight='bold')
        plt.legend(('Predicted Value', 'Actual Value'),
                   loc='upper right',fontsize = 'x-large')
    
    
        plt.show()

        final = final + 25
        
        if final > Length_of_TestData:
            final = Length_of_TestData
        
        if final>Count:  #will plot for the first 50 Samples
            break

# Saving Results in files

In [ ]:
def Save_Results_to_Files(Model_Name,Actual_Values,PredictedValues,Time_needed):

    Abs_Loss = []
    for i in range(0,len(Actual_Values)):
        Abs_Loss.append(abs(PredictedValues[i] - Actual_Values[i]))

    Abs_Loss = np.array(Abs_Loss)
    PredictedValues = np.array(PredictedValues)
    Actual_Values=np.array(Actual_Values)
    Loss_in_Percentage = []



    Predicted_Fuel_Consumption = pd.DataFrame({"Predicted_Values": PredictedValues,
                                                "Actual_Values":Actual_Values,
                                               "Abs_Loss":Abs_Loss,
                                                "Time_needed":Time_needed})

    CSV_Name_Loss = f'{Model_Name}_Predicted_Fuel_Consumption_for_{Vehicle_Type}.csv'
    Predicted_Fuel_Consumption.to_csv(f'{Folder}/Results/Electric/{CSV_Name_Loss}',index=False)
    print('\n\n***********CSV Saved************\n')

# Prediction with Different Network

In [ ]:
Models = ['DecisionTree','SimpleFeedForward','LinearRegression']

Actual_Values = y_test

for Model_Name in Models:

    if Model_Name == 'LinearRegression':
        print('\n\n\n*************Executing LR*************\n\n')
        PredictedValues = LinearRegression_Model()

    if Model_Name == 'DecisionTree':
        print('\n\n\n*************Executing DT*************\n\n')
        PredictedValues = DecisionTree_Model()

    if Model_Name == 'SimpleFeedForward':
        print('\n\n\n*************Executing FF*************\n\n')
        number_of_iteration = 800
        PredictedValues_FF = SimpleFeedForward_Model(number_of_iteration)
        Model_Name = 'SimpleFeedForward'
        PredictedValues = []
        for i in PredictedValues_FF:
            for j in i:
                PredictedValues.append(j)
        print(len(PredictedValues))

    ## Plot outputs
    Count = 50             #Indicates Plotting for how many samples
    Plotting_Pred_Actual(Model_Name,Actual_Values,PredictedValues,Length_of_TestData,Count)


    ## Creating a CSV with the predicted values

    Save_Results_to_Files(Model_Name,Actual_Values,PredictedValues,Time_needed)

